<a href="https://colab.research.google.com/github/xiashenzhen/geemap/blob/main/%E8%AE%AD%E7%BB%83%E6%9C%AC%E5%9C%B0%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用本地训练的模型在GEE分类

In [20]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-jason2024')
geemap.ee_initialize()

In [4]:
import pandas as pd
from geemap import ml
from sklearn import ensemble

In [ ]:
url = "https://raw.githubusercontent.com/gee-community/geemap/master/examples/data/rf_example.csv"
df = pd.read_csv(url)
df

In [6]:
feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = "landcover"

In [7]:
X = df[feature_names]
y = df[label]
n_trees = 10
rf = ensemble.RandomForestClassifier(n_trees).fit(X, y)

In [8]:
trees = ml.rf_to_strings(rf, feature_names)

In [9]:
print(len(trees))

10


In [ ]:
print(trees[0])

In [ ]:
ee_classifier = ml.strings_to_classifier(trees)
ee_classifier.getInfo()

In [12]:
# Make a cloud-free Landsat 8 TOA composite (from raw imagery).
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(
    collection=l8.filterDate('2018-01-01', '2018-12-31'), asFloat=True
)

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:204: DeprecationWarning: 

Attention required for LANDSAT/LC08/C01/T1! You are using a deprecated asset.
To ensure continued functionality, please update it by July 1, 2024.
Learn more: https://developers.google.com/earth-engine/landsat_c1_to_c2

  warnings.warn(warning, category=DeprecationWarning)


In [13]:
classified = image.select(feature_names).classify(ee_classifier)

In [ ]:
Map = geemap.Map(center=(37.75, -122.25), zoom=11)

Map.addLayer(
    image,
    {"bands": ['B7', 'B5', 'B3'], "min": 0.05, "max": 0.55, "gamma": 1.5},
    'image',
)
Map.addLayer(
    classified,
    {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},
    'classification',
)
Map

In [30]:
# id 不同，目前无法保存到云端
# asset_id = "ee-jason" + "/random_forest_strings_test"
# ml.export_trees_to_fc(trees, asset_id)

'ee-jason/random_forest_strings_test'

In [32]:
out_csv = "trees.csv"
ml.trees_to_csv(trees, out_csv)
another_classifier = ml.csv_to_classifier(out_csv)
classified = image.select(feature_names).classify(another_classifier)

In [ ]:
! pip install sankee

In [35]:
import sankee

sankee.datasets.LCMS_LC.sankify(
    years=[1990, 2000, 2010, 2020],
    region=ee.Geometry.Point([-122.192688, 46.25917]).buffer(2000),
    max_classes=3,
    title="Mount St. Helens Recovery",
)

SankeyPlot()

In [ ]:
Map = geemap.Map(height=650)
Map